In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
!pip install lightdock

In [ ]:
!mkdir test
!cd test
!wget https://raw.githubusercontent.com/lightdock/lightdock.github.io/master/tutorials/examples/2UUY/2UUY_rec.pdb
!wget https://raw.githubusercontent.com/lightdock/lightdock.github.io/master/tutorials/examples/2UUY/2UUY_lig.pdb

In [ ]:
!lightdock3_setup.py 2UUY_rec.pdb 2UUY_lig.pdb 1 10

In [ ]:
!lightdock3.py setup.json 10

In [ ]:
!head -2 swarm_0/gso_10.out

In [ ]:
import os
os.getcwd()
os.chdir('/kaggle/working/swarm_0')

In [ ]:
def Hello(str):
    for i in get_ipython().getoutput(str):
        print(i)
Hello('lgd_generate_conformations.py /kaggle/working/2UUY_rec.pdb /kaggle/working/2UUY_lig.pdb gso_10.out 10')


# complex protein-protein docking with restraints

In [ ]:
import os
os.chdir('/kaggle/working')
!mkdir complex_protein_protein
os.chdir('/kaggle/working/complex_protein_protein')
!wget https://raw.githubusercontent.com/lightdock/lightdock.github.io/master/tutorials/examples/4G6M/4G6M_rec.pdb
!wget https://raw.githubusercontent.com/lightdock/lightdock.github.io/master/tutorials/examples/4G6M/4G6M_lig.pdb


In [ ]:
!wget https://raw.githubusercontent.com/lightdock/lightdock.github.io/master/tutorials/examples/4G6M/restraints.list


In [ ]:
!lightdock3_setup.py 4G6M_rec.pdb 4G6M_lig.pdb 400 200 -anm --noxt --noh -rst restraints.list

In [ ]:
!lightdock3.py setup.json 50 -s fastdfire -c 2

In [ ]:
swarm_files = []

def execute(str):
    for i in get_ipython().getoutput(str):
        print(i)


root = "/kaggle/working/complex_protein_protein"
files = os.listdir(root)
for name in files:
    if os.path.isdir(root+"/"+name):
        if "swarm_" in name:
            swarm_files.append(root+"/"+name)


In [ ]:
swarm_files

In [ ]:
for swarm_f in swarm_files:
    str = 'lgd_generate_conformations.py '+root+'/4G6M_rec.pdb '+root+"/4G6M_lig.pdb "+swarm_f+"/gso_50.out 200"
    execute(str)

for swarm_f in swarm_files:
    str = 'lgd_cluster_bsas.py '+swarm_f+'/gso_100.out'
    execute(str)


In [ ]:
### Generate LightDock models ###

execute("ant_thony.py -c 8 generate_lightdock.list")

### Clustering BSAS (rmsd) within swarm ###

execute("ant_thony.py -c 8 cluster_lightdock.list")


In [ ]:
### Generate ranking files for filtering ###

execute("lgd_rank.py "+len(swarm_files)+" 100")

### Filtering models by >40% of satisfied restraints ### 

execute("lgd_filter_restraints.py --cutoff 5.0 --fnat 0.4 rank_by_scoring.list restraints.list")


## protein DNA docking example

In [ ]:
!mkdir test_protein_dna
!cd test_protein_dna
!wget https://raw.githubusercontent.com/lightdock/lightdock.github.io/master/tutorials/examples/1AZP/1AZP_A.pdb
!wget https://raw.githubusercontent.com/lightdock/lightdock.github.io/master/tutorials/examples/1AZP/1AZP_B.pdb

In [ ]:
!wget https://raw.githubusercontent.com/lightdock/lightdock.github.io/master/tutorials/examples/1AZP/restraints.list
!cat restraints.list

In [ ]:
!reduce -Trim 1AZP_A.pdb > 1AZP_A_noh.pdb
!reduce -BUILD 1AZP_A_noh.pdb > 1AZP_A_h.pdb

In [ ]:
!lightdock3_setup.py 1AZP_A_h.pdb 1AZP_B.pdb 400 200 -anm -rst restraints.list

In [ ]:
!lightdock3.py setup.json 100 -s dna -c 8